In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pylab as plt
import json
import time
import warnings

from bs4 import BeautifulSoup
from tqdm import tqdm
import requests

from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib
import plotly.express as px
from tqdm.autonotebook import tqdm

import openpyxl 
import pycountry

import nltk
#nltk.download()


warnings.filterwarnings('ignore')


c:\Users\javir\anaconda3\envs\ETL\lib\site-packages\tqdm_joblib\__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
titles = pd.read_csv('../data/clean/titles.csv', encoding='utf-8', encoding_errors='ignore')
comments = pd.read_csv('../data/clean/com_group.csv', encoding='utf-8', encoding_errors='ignore')

## Test

In [3]:
# porcentaje de nulos de la tabla Titles
nan_cols = titles.isna().mean() * 100 
nan_cols[nan_cols>0]

Series([], dtype: float64)

In [4]:
# porcentaje de nulos de la tabla Titles
nan_cols = comments.isna().mean() * 100 
nan_cols[nan_cols>0]

title    2.233351
dtype: float64

In [5]:
pregunta1= 'Contemporáneas'
#pregunta2= 'Acción'
pregunta3= 'Misterio'
pregunta4= ''
pregunta5= 'Hechos reales'
pregunta6= ('0','300')
pregunta7= 'Mucho'
pregunta8= 'hero'
pregunta9= 'Netflix'



In [6]:
respuestas = {
    'pregunta1': pregunta1,
    #'pregunta2': pregunta2,
    'pregunta3': pregunta3,
    'pregunta4': pregunta4,
    'pregunta5': pregunta5,
    #'pregunta6': (duracion_minima, duracion_maxima),
    'pregunta7': pregunta7,
    'pregunta8': pregunta8,
    'pregunta9': pregunta9,
}

In [7]:
# 1. Filtro por antiguedad
if respuestas['pregunta1'] == 'Clásicas':
    df_filtrado = titles[titles['release_year'] <= 1990]
elif respuestas['pregunta1'] == 'Contemporáneas':
    df_filtrado = titles[titles['release_year'] > 1990]
elif respuestas['pregunta1'] == '¯\_(ツ)_/¯':
    df_filtrado = titles

In [8]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
84,ts20981,Survivor,SHOW,A reality show contest where sixteen or more c...,2000,TV-PG,48,reality,United States,42.0,tt0239195,7.4,24687,5419,7.4,Netflix,Jeff Probst
85,tm107744,Mission: Impossible,MOVIE,"When Ethan Hunt, the leader of a crack espiona...",1996,PG-13,110,"thriller, action",United States,-1.0,tt0117060,7.1,420673,41115,6.9,Netflix,"Tom Cruise, Jon Voight, Henry Czerny, Emmanuel..."
86,ts22176,Stargate SG-1,SHOW,The story of Stargate SG-1 begins about a year...,1997,TV-PG,44,"scifi, drama, action","Canada, United States",10.0,tt0118480,8.4,90196,88851,8.3,"Netflix, Amazon","Ben Browder, Amanda Tapping, Christopher Judge..."
87,tm15897,Se7en,MOVIE,Two homicide detectives are on a desperate hun...,1995,R,127,"crime, thriller, drama",United States,-1.0,tt0114369,8.6,1606270,51633,8.4,"Netflix, HBO","Brad Pitt, Morgan Freeman, Gwyneth Paltrow, Ke..."
88,ts21223,Pokémon,SHOW,"Join Ash accompanied by his partner Pikachu, a...",1997,TV-Y7,22,"action, scifi, animation, comedy, fantasy, family",Japan,25.0,tt14053732,7.4,198,115356,7.6,"Netflix, Amazon","Rica Matsumoto, Ikue Otani, Daiki Yamashita, K..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15311,tm948025,Hard Luck Love Song,MOVIE,"Jesse, a charismatic but down on his luck trou...",2023,R,104,"drama, thriller, romance, music",United States,-1.0,tt8709338,5.3,444,4299,5.3,Amazon,"Michael Dorman, Sophia Bush, RZA, Dermot Mulro..."
15312,tm811070,The Man on the Other Side,MOVIE,"It is 1974, and in the Cold War paranoia of Ea...",2023,R,85,"action, drama, thriller","United Kingdom, Singapore, Germany",-1.0,tt8000096,4.6,189,3415,2.0,Amazon,"Mari Bensel, James Carney, Nils Schulz, Jörg M..."
15313,ts382337,Marc Marquez: ALL IN,SHOW,Marc Márquez is facing the toughest decision o...,2023,unknown,33,documentation,Spain,1.0,tt26762647,8.1,74,2893,6.0,Amazon,"Marc Márquez, Dani Pedrosa, Jorge Lorenzo, Ale..."
15314,tm1292884,Hunt,MOVIE,"Arjun is the ACP of Hyderabad, who gets involv...",2023,R,129,"action, thriller, crime",India,-1.0,tt21946058,5.8,1269,4403,4.3,Amazon,"Sudheer Babu, Srikanth Meka, Bharath Srinivasa..."


In [10]:
# 2. Filtro por género favorito
if respuestas['pregunta2'] == 'Acción':
    df_filtrado = df_filtrado[df_filtrado['genres'].str.contains('action', case=False)]
elif respuestas['pregunta2'] == 'Drama':
    df_filtrado = df_filtrado[df_filtrado['genres'].str.contains('drama', case=False)]
elif respuestas['pregunta2'] == 'Comedia':
    df_filtrado = df_filtrado[df_filtrado['genres'].str.contains('comedy', case=False)]
elif respuestas['pregunta2'] == 'Documental':
    df_filtrado = df_filtrado[df_filtrado['genres'].str.contains('documentation', case=False)]
elif respuestas['pregunta2'] == 'Horror':
    df_filtrado = df_filtrado[df_filtrado['genres'].str.contains('horror', case=False) | df_filtrado['genres'].str.contains('thriller', case=False)]
elif respuestas['pregunta2'] == 'Romance':
    df_filtrado = df_filtrado[df_filtrado['genres'].str.contains('romance', case=False)]
elif respuestas['pregunta2'] == '¯\_(ツ)_/¯':
    pass

In [9]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
84,ts20981,Survivor,SHOW,A reality show contest where sixteen or more c...,2000,TV-PG,48,reality,United States,42.0,tt0239195,7.4,24687,5419,7.4,Netflix,Jeff Probst
85,tm107744,Mission: Impossible,MOVIE,"When Ethan Hunt, the leader of a crack espiona...",1996,PG-13,110,"thriller, action",United States,-1.0,tt0117060,7.1,420673,41115,6.9,Netflix,"Tom Cruise, Jon Voight, Henry Czerny, Emmanuel..."
86,ts22176,Stargate SG-1,SHOW,The story of Stargate SG-1 begins about a year...,1997,TV-PG,44,"scifi, drama, action","Canada, United States",10.0,tt0118480,8.4,90196,88851,8.3,"Netflix, Amazon","Ben Browder, Amanda Tapping, Christopher Judge..."
87,tm15897,Se7en,MOVIE,Two homicide detectives are on a desperate hun...,1995,R,127,"crime, thriller, drama",United States,-1.0,tt0114369,8.6,1606270,51633,8.4,"Netflix, HBO","Brad Pitt, Morgan Freeman, Gwyneth Paltrow, Ke..."
88,ts21223,Pokémon,SHOW,"Join Ash accompanied by his partner Pikachu, a...",1997,TV-Y7,22,"action, scifi, animation, comedy, fantasy, family",Japan,25.0,tt14053732,7.4,198,115356,7.6,"Netflix, Amazon","Rica Matsumoto, Ikue Otani, Daiki Yamashita, K..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15311,tm948025,Hard Luck Love Song,MOVIE,"Jesse, a charismatic but down on his luck trou...",2023,R,104,"drama, thriller, romance, music",United States,-1.0,tt8709338,5.3,444,4299,5.3,Amazon,"Michael Dorman, Sophia Bush, RZA, Dermot Mulro..."
15312,tm811070,The Man on the Other Side,MOVIE,"It is 1974, and in the Cold War paranoia of Ea...",2023,R,85,"action, drama, thriller","United Kingdom, Singapore, Germany",-1.0,tt8000096,4.6,189,3415,2.0,Amazon,"Mari Bensel, James Carney, Nils Schulz, Jörg M..."
15313,ts382337,Marc Marquez: ALL IN,SHOW,Marc Márquez is facing the toughest decision o...,2023,unknown,33,documentation,Spain,1.0,tt26762647,8.1,74,2893,6.0,Amazon,"Marc Márquez, Dani Pedrosa, Jorge Lorenzo, Ale..."
15314,tm1292884,Hunt,MOVIE,"Arjun is the ACP of Hyderabad, who gets involv...",2023,R,129,"action, thriller, crime",India,-1.0,tt21946058,5.8,1269,4403,4.3,Amazon,"Sudheer Babu, Srikanth Meka, Bharath Srinivasa..."


In [10]:
# 3. Filtro por tipo de trama favorita
sinonimos_misterio = ['mystery', 'enigma', 'puzzle', 'riddle', 'conundrum', 'secret', 'intrigue', 'clue', 'suspense']
sinonimos_aventura = ['thrill', 'adventure', 'expedition', 'journey', 'quest', 'explor', 'trek', 'voyage', 'clue', 'safari', 'exploration']
sinonimos_fantasia = ['fantasy', 'imagination', 'imaginary', 'enchant', 'magic', 'tail', 'fairy', 'myth', 'wonder', 'dream', 'illusion', 'super', 'superhero']

def buscar_sinonimos(critica, sinonimos):
    for sinonimo in sinonimos:
        if sinonimo in critica:
            return True
    return False

if respuestas['pregunta3'] == 'Misterio':
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_misterio))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta3'] == 'Aventura':
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_aventura))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta3'] == 'Fantasía':
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_fantasia))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta3'] == '¯\_(ツ)_/¯':
    pass

In [11]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,ts20981,Survivor,SHOW,A reality show contest where sixteen or more c...,2000,TV-PG,48,reality,United States,42.0,tt0239195,7.4,24687,5419,7.4,Netflix,Jeff Probst
1,tm107744,Mission: Impossible,MOVIE,"When Ethan Hunt, the leader of a crack espiona...",1996,PG-13,110,"thriller, action",United States,-1.0,tt0117060,7.1,420673,41115,6.9,Netflix,"Tom Cruise, Jon Voight, Henry Czerny, Emmanuel..."
2,tm15897,Se7en,MOVIE,Two homicide detectives are on a desperate hun...,1995,R,127,"crime, thriller, drama",United States,-1.0,tt0114369,8.6,1606270,51633,8.4,"Netflix, HBO","Brad Pitt, Morgan Freeman, Gwyneth Paltrow, Ke..."
3,tm191013,Boogie Nights,MOVIE,"Set in 1977, back when sex was safe, pleasure ...",1997,R,156,drama,United States,-1.0,tt0118749,7.9,261563,20436,7.6,"Netflix, HBO","Mark Wahlberg, Julianne Moore, Burt Reynolds, ..."
4,tm122434,Forrest Gump,MOVIE,A man with a low IQ has accomplished great thi...,1994,PG-13,142,"drama, romance",United States,-1.0,tt0109830,8.8,2021343,63449,8.5,Netflix,"Tom Hanks, Robin Wright, Gary Sinise, Mykelti ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6273,tm1287099,Ten Little Mistresses,MOVIE,"A widowed billionaire, Valentin Esposo, and hi...",2023,unknown,125,comedy,Philippines,-1.0,tt24787466,6.2,142,8526,5.7,Amazon,"John Arcilla, Eugene Domingo, Pokwang, Carmi M..."
6274,ts365330,Cinema Marte Dum Tak,SHOW,"J Neelam, Vinod Talwar, Dilip Gulati and Kisha...",2023,unknown,37,documentation,India,1.0,tt21908844,7.6,335,2457,2.0,Amazon,"Vinod Talwar, Dilip Gulati, Kishan Shah, Jaspa..."
6275,tm1302281,Christopher,MOVIE,"A maverick, vigilante cop who is forced to mis...",2023,R,150,"thriller, action, crime",India,-1.0,tt21819994,7.3,1708,362,8.0,Amazon,"Mammootty, Sneha, Amala Paul, Aishwarya Lekshm..."
6276,tm1294232,Thankam,MOVIE,Muthu and Kannan are Gold Agents from Thrissur...,2023,unknown,145,"drama, crime",India,-1.0,tt11086194,7.3,2868,5722,8.8,Amazon,"Biju Menon, Vineeth Sreenivasan, Aparna Balamu..."


In [12]:
# 4. Filtro por actor o actriz favorito
if respuestas['pregunta4'] != '':
    actor_favorito = respuestas['pregunta4']
    df_filtrado = df_filtrado[df_filtrado['actors'].str.contains(actor_favorito, case=False)]
else:
    pass

In [13]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,ts20981,Survivor,SHOW,A reality show contest where sixteen or more c...,2000,TV-PG,48,reality,United States,42.0,tt0239195,7.4,24687,5419,7.4,Netflix,Jeff Probst
1,tm107744,Mission: Impossible,MOVIE,"When Ethan Hunt, the leader of a crack espiona...",1996,PG-13,110,"thriller, action",United States,-1.0,tt0117060,7.1,420673,41115,6.9,Netflix,"Tom Cruise, Jon Voight, Henry Czerny, Emmanuel..."
2,tm15897,Se7en,MOVIE,Two homicide detectives are on a desperate hun...,1995,R,127,"crime, thriller, drama",United States,-1.0,tt0114369,8.6,1606270,51633,8.4,"Netflix, HBO","Brad Pitt, Morgan Freeman, Gwyneth Paltrow, Ke..."
3,tm191013,Boogie Nights,MOVIE,"Set in 1977, back when sex was safe, pleasure ...",1997,R,156,drama,United States,-1.0,tt0118749,7.9,261563,20436,7.6,"Netflix, HBO","Mark Wahlberg, Julianne Moore, Burt Reynolds, ..."
4,tm122434,Forrest Gump,MOVIE,A man with a low IQ has accomplished great thi...,1994,PG-13,142,"drama, romance",United States,-1.0,tt0109830,8.8,2021343,63449,8.5,Netflix,"Tom Hanks, Robin Wright, Gary Sinise, Mykelti ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6273,tm1287099,Ten Little Mistresses,MOVIE,"A widowed billionaire, Valentin Esposo, and hi...",2023,unknown,125,comedy,Philippines,-1.0,tt24787466,6.2,142,8526,5.7,Amazon,"John Arcilla, Eugene Domingo, Pokwang, Carmi M..."
6274,ts365330,Cinema Marte Dum Tak,SHOW,"J Neelam, Vinod Talwar, Dilip Gulati and Kisha...",2023,unknown,37,documentation,India,1.0,tt21908844,7.6,335,2457,2.0,Amazon,"Vinod Talwar, Dilip Gulati, Kishan Shah, Jaspa..."
6275,tm1302281,Christopher,MOVIE,"A maverick, vigilante cop who is forced to mis...",2023,R,150,"thriller, action, crime",India,-1.0,tt21819994,7.3,1708,362,8.0,Amazon,"Mammootty, Sneha, Amala Paul, Aishwarya Lekshm..."
6276,tm1294232,Thankam,MOVIE,Muthu and Kannan are Gold Agents from Thrissur...,2023,unknown,145,"drama, crime",India,-1.0,tt11086194,7.3,2868,5722,8.8,Amazon,"Biju Menon, Vineeth Sreenivasan, Aparna Balamu..."


In [14]:

# 6. Filtro para determinar ficción
palabras_clave_ficcion = ['fiction', 'imaginary', 'fantasy', 'fictional', 'otherworld', 'extraterrestrial']
    
if respuestas['pregunta5'] == "Ficción":
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, palabras_clave_ficcion))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta5'] == "Hechos reales":
    comentarios_filtrados = comments[~comments['review'].apply(lambda x: buscar_sinonimos(x, palabras_clave_ficcion))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta5'] == '¯\_(ツ)_/¯':
    pass


In [15]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,ts20981,Survivor,SHOW,A reality show contest where sixteen or more c...,2000,TV-PG,48,reality,United States,42.0,tt0239195,7.4,24687,5419,7.4,Netflix,Jeff Probst
1,tm122434,Forrest Gump,MOVIE,A man with a low IQ has accomplished great thi...,1994,PG-13,142,"drama, romance",United States,-1.0,tt0109830,8.8,2021343,63449,8.5,Netflix,"Tom Hanks, Robin Wright, Gary Sinise, Mykelti ..."
2,tm25947,Snatch,MOVIE,"Unscrupulous boxing promoters, violent bookmak...",2000,R,103,"crime, comedy","United States, United Kingdom",-1.0,tt0208092,8.3,841435,25203,7.8,Netflix,"Benicio del Toro, Dennis Farina, Vinnie Jones,..."
3,tm43808,The Talented Mr. Ripley,MOVIE,Tom Ripley is a calculating young man who beli...,1999,R,139,"thriller, drama, crime",United States,-1.0,tt0134119,7.4,207524,1378,7.2,"Netflix, Amazon","Matt Damon, Gwyneth Paltrow, Jude Law, Cate Bl..."
4,tm27395,Mission: Impossible II,MOVIE,With computer genius Luther Stickell at his si...,2000,PG-13,123,"thriller, action",United States,-1.0,tt0120755,6.1,337987,29392,6.1,"Netflix, Amazon","Tom Cruise, Dougray Scott, Thandiwe Newton, Vi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3828,tm1287099,Ten Little Mistresses,MOVIE,"A widowed billionaire, Valentin Esposo, and hi...",2023,unknown,125,comedy,Philippines,-1.0,tt24787466,6.2,142,8526,5.7,Amazon,"John Arcilla, Eugene Domingo, Pokwang, Carmi M..."
3829,ts365330,Cinema Marte Dum Tak,SHOW,"J Neelam, Vinod Talwar, Dilip Gulati and Kisha...",2023,unknown,37,documentation,India,1.0,tt21908844,7.6,335,2457,2.0,Amazon,"Vinod Talwar, Dilip Gulati, Kishan Shah, Jaspa..."
3830,tm1302281,Christopher,MOVIE,"A maverick, vigilante cop who is forced to mis...",2023,R,150,"thriller, action, crime",India,-1.0,tt21819994,7.3,1708,362,8.0,Amazon,"Mammootty, Sneha, Amala Paul, Aishwarya Lekshm..."
3831,tm1294232,Thankam,MOVIE,Muthu and Kannan are Gold Agents from Thrissur...,2023,unknown,145,"drama, crime",India,-1.0,tt11086194,7.3,2868,5722,8.8,Amazon,"Biju Menon, Vineeth Sreenivasan, Aparna Balamu..."


In [16]:
# 6. Filtro por duración de la película
#df_filtrado = df_filtrado[(df_filtrado['duration'] >= duracion_minima) & (df_filtrado['duration'] <= duracion_maxima)]

In [17]:
# 7. Filtro por final feliz
sinonimos_final_feliz = ['happy ending', 'positive outcome', 'pleasant conclusion', 'satisfying resolution', 'joyful finale', 'contented ending', 'delightful outcome', 'cheerful conclusion', 'pleasant ending', 'uplifting finale']
    
if respuestas['pregunta7'] == 'Mucho':
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_final_feliz))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta7'] == 'No':
    comentarios_filtrados = comments[~comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_final_feliz))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta7'] == '¯\_(ツ)_/¯':
    pass

In [18]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm43808,The Talented Mr. Ripley,MOVIE,Tom Ripley is a calculating young man who beli...,1999,R,139,"thriller, drama, crime",United States,-1.0,tt0134119,7.4,207524,1378,7.2,"Netflix, Amazon","Matt Damon, Gwyneth Paltrow, Jude Law, Cate Bl..."
1,tm116488,Natural Born Killers,MOVIE,Two victims of traumatized childhoods become l...,1994,R,118,"thriller, drama, crime, action",United States,-1.0,tt0110632,7.3,11,21933,7.1,Netflix,"Woody Harrelson, Juliette Lewis, Robert Downey..."
2,tm56574,You've Got Mail,MOVIE,"Book superstore magnate, Joe Fox and independe...",1998,PG,119,"comedy, romance, drama",United States,-1.0,tt0128853,6.7,211635,21724,6.6,"Netflix, HBO","Tom Hanks, Meg Ryan, Greg Kinnear, Parker Pose..."
3,tm53991,Big Daddy,MOVIE,A lazy law school grad adopts a kid to impress...,1999,PG-13,93,"drama, comedy, european",United States,-1.0,tt0142342,6.4,215053,26848,6.4,"Netflix, Amazon","Adam Sandler, Joey Lauren Adams, Jon Stewart, ..."
4,tm55100,Dil Se..,MOVIE,Journalist Amar falls for a mysterious woman o...,1998,PG-13,163,"drama, romance",India,-1.0,tt0164538,7.5,28586,7811,6.8,Netflix,"Shah Rukh Khan, Manisha Koirala, Preity Zinta,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,tm552220,The Blackout Experiment,MOVIE,6 strangers wake up inside a room garnished wi...,2021,R,81,"horror, thriller, drama",United States,-1.0,tt3659284,2.9,239,6395,5.0,Amazon,"Yasmin Irvine, Stefani Rose, Uli Auliani, Chev..."
294,tm822572,Gun and a Hotel Bible,MOVIE,"Based on the original play, Gun and a Hotel Bi...",2021,unknown,58,drama,United States,-1.0,tt10661500,4.3,145,6,6.3,Amazon,"Bradley Gosnell, Daniel Floren, Mia Marcon, Al..."
295,tm573955,Spoken,MOVIE,A group of friends try to find balance in thei...,2021,R,74,"drama, romance, horror, thriller",United States,-1.0,tt6455590,2.4,150,147,4.0,Amazon,"Ryan Paynter, Steven Serbones, Chelsea Skalski..."
296,tm1071643,"Bigfoot, UFOs and Jesus",MOVIE,After her estranged father dies during a UFO s...,2021,unknown,98,"drama, family, scifi, comedy",unknown,-1.0,tt14831408,3.6,162,704,5.0,Amazon,"Melissa Anschutz, Don Most, Victoria Jackson, ..."


In [19]:
# 8. Filtro por tema, época o lugar favorito
def buscar_sinonimos(critica, sinonimos):
    for sinonimo in sinonimos:
        if sinonimo in critica:
            return True
    return False
    
if respuestas['pregunta8'] != '':
    tema = respuestas['pregunta8']
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, tema))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
else:
    pass

In [20]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm43808,The Talented Mr. Ripley,MOVIE,Tom Ripley is a calculating young man who beli...,1999,R,139,"thriller, drama, crime",United States,-1.0,tt0134119,7.4,207524,1378,7.2,"Netflix, Amazon","Matt Damon, Gwyneth Paltrow, Jude Law, Cate Bl..."
1,tm116488,Natural Born Killers,MOVIE,Two victims of traumatized childhoods become l...,1994,R,118,"thriller, drama, crime, action",United States,-1.0,tt0110632,7.3,11,21933,7.1,Netflix,"Woody Harrelson, Juliette Lewis, Robert Downey..."
2,tm56574,You've Got Mail,MOVIE,"Book superstore magnate, Joe Fox and independe...",1998,PG,119,"comedy, romance, drama",United States,-1.0,tt0128853,6.7,211635,21724,6.6,"Netflix, HBO","Tom Hanks, Meg Ryan, Greg Kinnear, Parker Pose..."
3,tm53991,Big Daddy,MOVIE,A lazy law school grad adopts a kid to impress...,1999,PG-13,93,"drama, comedy, european",United States,-1.0,tt0142342,6.4,215053,26848,6.4,"Netflix, Amazon","Adam Sandler, Joey Lauren Adams, Jon Stewart, ..."
4,tm55100,Dil Se..,MOVIE,Journalist Amar falls for a mysterious woman o...,1998,PG-13,163,"drama, romance",India,-1.0,tt0164538,7.5,28586,7811,6.8,Netflix,"Shah Rukh Khan, Manisha Koirala, Preity Zinta,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,tm552220,The Blackout Experiment,MOVIE,6 strangers wake up inside a room garnished wi...,2021,R,81,"horror, thriller, drama",United States,-1.0,tt3659284,2.9,239,6395,5.0,Amazon,"Yasmin Irvine, Stefani Rose, Uli Auliani, Chev..."
294,tm822572,Gun and a Hotel Bible,MOVIE,"Based on the original play, Gun and a Hotel Bi...",2021,unknown,58,drama,United States,-1.0,tt10661500,4.3,145,6,6.3,Amazon,"Bradley Gosnell, Daniel Floren, Mia Marcon, Al..."
295,tm573955,Spoken,MOVIE,A group of friends try to find balance in thei...,2021,R,74,"drama, romance, horror, thriller",United States,-1.0,tt6455590,2.4,150,147,4.0,Amazon,"Ryan Paynter, Steven Serbones, Chelsea Skalski..."
296,tm1071643,"Bigfoot, UFOs and Jesus",MOVIE,After her estranged father dies during a UFO s...,2021,unknown,98,"drama, family, scifi, comedy",unknown,-1.0,tt14831408,3.6,162,704,5.0,Amazon,"Melissa Anschutz, Don Most, Victoria Jackson, ..."


In [21]:
# 9. Filtro por plataforma
if respuestas['pregunta9'] != '':
    plat_favorito = respuestas['pregunta9']
    df_filtrado = df_filtrado[df_filtrado['platform'].str.contains(plat_favorito, case=False)]
else:
    pass


In [22]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm43808,The Talented Mr. Ripley,MOVIE,Tom Ripley is a calculating young man who beli...,1999,R,139,"thriller, drama, crime",United States,-1.0,tt0134119,7.4,207524,1378,7.2,"Netflix, Amazon","Matt Damon, Gwyneth Paltrow, Jude Law, Cate Bl..."
1,tm116488,Natural Born Killers,MOVIE,Two victims of traumatized childhoods become l...,1994,R,118,"thriller, drama, crime, action",United States,-1.0,tt0110632,7.3,11,21933,7.1,Netflix,"Woody Harrelson, Juliette Lewis, Robert Downey..."
2,tm56574,You've Got Mail,MOVIE,"Book superstore magnate, Joe Fox and independe...",1998,PG,119,"comedy, romance, drama",United States,-1.0,tt0128853,6.7,211635,21724,6.6,"Netflix, HBO","Tom Hanks, Meg Ryan, Greg Kinnear, Parker Pose..."
3,tm53991,Big Daddy,MOVIE,A lazy law school grad adopts a kid to impress...,1999,PG-13,93,"drama, comedy, european",United States,-1.0,tt0142342,6.4,215053,26848,6.4,"Netflix, Amazon","Adam Sandler, Joey Lauren Adams, Jon Stewart, ..."
4,tm55100,Dil Se..,MOVIE,Journalist Amar falls for a mysterious woman o...,1998,PG-13,163,"drama, romance",India,-1.0,tt0164538,7.5,28586,7811,6.8,Netflix,"Shah Rukh Khan, Manisha Koirala, Preity Zinta,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,tm361414,The Guilty,MOVIE,"Police officer Asger Holm, demoted to desk wor...",2018,R,85,"thriller, crime, drama, european",Denmark,-1.0,tt6742252,7.5,60804,12353,7.3,"Netflix, HBO","Jakob Cedergren, Jessica Dinnage, Omar Shargaw..."
228,tm228223,The Innocents,MOVIE,"Poland, 1945. Mathilde, a young French Red Cro...",2016,PG-13,115,"drama, history, european","Belgium, France, Poland",-1.0,tt4370784,7.3,10518,10035,7.1,"Netflix, Amazon","Lou de Laâge, Agata Buzek, Agata Kulesza, Vinc..."
244,tm317586,Paradox,MOVIE,When a Hong Kong police negotiator is informed...,2017,R,101,"action, drama, crime, thriller","China, Hong Kong",-1.0,tt7245176,6.5,4109,18504,6.6,"Netflix, Amazon","Louis Koo, Wu Yue, Gordon Lam, Chris Collins, ..."
245,tm438262,Unstoppable,MOVIE,Dong-chul and Ji-soo are a happily married cou...,2018,R,115,"crime, thriller, action, european","Korea, Republic of",-1.0,tt9225192,6.6,3740,13161,7.6,"Netflix, Amazon","Ma Dong-seok, Song Ji-hyo, Kim Sung-oh, Kim Mi..."


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
# Definir objeto vectorizador TF_IDF

tfidf = TfidfVectorizer(stop_words= 'english')

In [25]:
# contruir matriz TF-IDF

tfidf_matrix = tfidf.fit_transform(df_filtrado['description'])
tfidf_matrix.shape

(133, 1720)

In [26]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim= linear_kernel(tfidf_matrix, tfidf_matrix)

In [27]:
# construir mapa inverso de indices y titulos de peliculas

indices= pd.Series(df_filtrado.index, index= df_filtrado['title']).drop_duplicates()

In [28]:
def get_recom (title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores= list(enumerate(cosine_sim[idx]))

    sim_scores= sorted(sim_scores, key= lambda x: x[1], reverse=True)

    sim_scores=  sim_scores[1:11]

    movie_id= [i[0]for i in sim_scores]

    df_filtrado_sorted = df_filtrado.iloc[movie_id].sort_values(by='imdb_score', ascending=False)

    return df_filtrado_sorted['title']

In [29]:
get_recom('Natural Born Killers')

123            Newton's Cradle
4                     Dil Se..
0      The Talented Mr. Ripley
5                      Monster
2              You've Got Mail
113        42 Days of Darkness
3                    Big Daddy
97           The Longest Night
120       Why Did You Kill Me?
6              Good Luck Chuck
Name: title, dtype: object